# svg_files

> Loading and manipulating SVG files

In [ ]:
#| default_exp svg_files

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
import numpy as np
import svgpathtools
from lxml import etree

from singleline_dataset.transforms import *

In [ ]:
#| hide
from nbdev.showdoc import *

## Converting to Strokes

Steps:

1. **Discretize the SVG:** Convert all paths (including bezier curves, etc) to a set of discrete points.
2. **Apply SVG Global Transforms:** If the SVG contains something like `<g transform="(scale:0.5,-0.4)">`, apply that to the coordinates.
3. **Rescale:** Find the min/max/range on the x and y axes. Subtract min, divide by range to get coords between 0 and 1. Multiply by `target_size` to get coordinates in a comprehensible range.

In [ ]:
#| exports
def svg_to_strokes(input_fname, target_size=200, total_n=1000, min_n=3):
    # convert all paths to a series of points
    all_strokes = discretize_svg(input_fname, total_n=total_n, min_n=min_n)
    # apply any global SVG transform instructions
    globally_rescaled_strokes = global_svg_transform(all_strokes, input_fname)
    # rescaled to the target image size
    rescaled_strokes = rescale_strokes(globally_rescaled_strokes, target_size)
    return rescaled_strokes

## Discretizing SVG Paths

In [ ]:
#| export
def to_points(parsed_path, n):
    prev = None
    acc = []
    step_size = 1.0 / n
    for x in np.arange(0.0, 1.0, step_size):
        _complex_pt = parsed_path.point(x)
        _x = _complex_pt.real
        _y = _complex_pt.imag
        pt = (_x, _y)
        acc.append(pt)
    return acc


def discretize_path(path, total_n=1000, min_n=3):
    chunk_size = path.length() / total_n
    subpaths = []
    for subpath in path.continuous_subpaths():
        sub_n = subpath.length() / chunk_size
        # drop paths shorter than the minimum number of points.
        if sub_n < min_n:
            continue
        subpaths.append(np.array(to_points(subpath, n=sub_n)))
    return np.array(subpaths, dtype=object)

In [ ]:
#| exports
def discretize_paths(paths, total_n=1000, min_n=3):
    all_strokes = []
    total_length = sum([p.length() for p in paths])
    for p in paths:
        # compute number of points to generate for curr path based on ratio to total path length
        path_n = int(total_n * p.length() / total_length)
        # if number of points is less than the minimum, drop it.
        if path_n < min_n:
            continue

        strokes = discretize_path(p, path_n, min_n=min_n)
        all_strokes.extend(strokes)
    return all_strokes


def discretize_svg(input_fname, **kwargs):
    paths, attributes, svg_attributes = svgpathtools.svg2paths(
        input_fname, return_svg_attributes=True
    )
    all_strokes = discretize_paths(paths, **kwargs)
    return all_strokes

## Rescale

In [ ]:
#| export
def rescale_strokes(all_strokes, target_size):
    # even though we want to default to keeping paths separate (until we can do bbox checking),
    # we do want a global max/min coord so that we can rescale all points within the same space.
    vstack_coords = np.vstack(all_strokes)
    # print(f"vstack_coords.shape={vstack_coords.shape}")
    vstack_bounding_box = BoundingBox.create(vstack_coords)
    vstack_rescale_transform = vstack_bounding_box.normalization_xform(target_size)

    all_rescaled_strokes = [
        apply_transform(c[:, :2], vstack_rescale_transform)[:, :2] for c in all_strokes
    ]
    return np.array(all_rescaled_strokes, dtype=object)

## Global Transforms

In [ ]:
#| export
def global_svg_transform(all_strokes, input_fname):
    global_transform = svg_to_transforms(parse_svg(input_fname))
    if not global_transform:
        return all_strokes
    transformed_strokes = []
    for path_strokes in all_strokes:
        strokes = [apply_transform(s, global_transform) for s in path_strokes]
        transformed_strokes.append(strokes)
    return transformed_strokes


def parse_svg(input_file):
    """Parse the SVG XML, to allow extracting individual elements"""
    parsed = etree.parse(input_file)
    return parsed.getroot()


def svg_to_transforms(svgroot):
    def _extract_g(svgroot):
        all_parsed = []
        all_g = svgroot.findall(".//{http://www.w3.org/2000/svg}g")
        for g in all_g:
            all_parsed.append(g.attrib)
        return all_parsed

    def _get_transform_str(svgroot):
        all_g = _extract_g(svgroot)
        if not all_g:
            # print('no <g> tags found')
            return None
        elif len(all_g) > 1:
            return Exception(
                f"unexpected number of <g> tags found in SVG: {len(all_g)}"
            )
        g = all_g[0]
        if "transform" not in g:
            # print('"transform" attribute not found in <g>')
            return None
        transform_str = g["transform"]
        return transform_str

    transform_str = _get_transform_str(svgroot)
    if not transform_str:
        return None

    return build_transforms(transform_str)


def parse_transform_instruction(item):
    def strip_tname(tname, item, expected_len):
        parts = item.replace(tname, "").replace("(", "").replace(")", "").split(",")
        nums = [float(n) for n in parts]
        if len(nums) != expected_len:
            raise Exception(
                f'unexpected length for transform instruction "{item}": {len(nums)} (expected {expected_len})'
            )
        return nums

    if item.startswith("translate"):
        tx, ty = strip_tname("translate", item, expected_len=2)
        # print(f"tx={tx}, ty={ty}")
        return translate_xform(tx, ty)
    elif item.startswith("rotate"):
        ang = strip_tname("rotate", item, expected_len=1)
        # print(f"ang={ang}")
        return rotate_xform(ang[0])
    elif item.startswith("scale"):
        sx, sy = strip_tname("scale", item, expected_len=2)
        # print(f"sx={sx}, sy={sy}")
        return scale_xform(sx, sy)
    else:
        raise Exception(f"runrecognized transform instruction: {item}")


def build_transforms(tstr):
    xforms = [parse_transform_instruction(part) for part in tstr.split(" ")]
    final = identity_xform()
    for x in reversed(xforms):
        final = final.dot(x)
    return final

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()